In [1]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import random

In [2]:
#Loading Variables
WIDTH = 1600
HEIGHT = 900
ORANGE_BACKGROUND = [12, 255, 255]
orange_signs_list = os.listdir('./Img/RawOrangeSignImgs/')
daily_templates_list = os.listdir('./Img/Templates/Daily')
frequent_templates_list = os.listdir('./Img/Templates/Frequent')
uncommom_templates_list = os.listdir('./Img/Templates/Uncommon') 
templates_list = [(daily_templates_list, frequent_templates_list, uncommom_templates_list)]   
coco_imgs_list = os.listdir('./coco_dataset/images/chosenImages')
#orange_signs_list = ['R-2_000.png']
#coco_imgs_list = ['000000000025.jpg']
coco_imgs_list = coco_imgs_list[:100]




In [3]:
def applyErosion(mask):
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    return mask

def getContours(mask):
    valid_contours_list = []
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        area = cv2.contourArea(c)
        if area < 600:
            continue
        valid_contours_list.append(c)
    return valid_contours_list

def createMask(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
    # print("HSV Img Shape: ", hsv_img.shape)

    lower_limit = np.array(ORANGE_BACKGROUND) #Background HSV Color to create mask
    upper_limit = np.array(ORANGE_BACKGROUND) #Background HSV Color to create mask

    mask = cv2.inRange(hsv_img, lower_limit, upper_limit) #Select orange area
    mask_inv = cv2.bitwise_not(mask)                      #Invert mask (deselect orang area)        
    mask_inv = applyErosion(mask_inv)                     #Erode the mask to remove noise
    #print("Mask-Inv Shape:" + mask_inv.shape)  #should be (h, w, 3)
    
    return mask_inv

def createTemplate(orange_signs_list):
    templates_dict = {}
    for name in orange_signs_list:
        name = name[:-4:] # remove .png
        sign_type = name[:-4:] # select the sign type
        sign_number = name[-4::] # select the number of this sign type
        img = cv2.imread(f'./Img/RawOrangeSignImgs/{sign_type + sign_number}.png')
        #cv2.imshow("Img", img)
        #print('Img Shape:', img.shape)     

        mask = createMask(img)
        contours_list = getContours(mask) #Needed to the annotations

        #Une a máscara invertida sobre a imagem principal para destacar a placa 
        sign = cv2.bitwise_and(img, img, mask= mask)
        sign_rgba = cv2.cvtColor(sign, cv2.COLOR_BGR2BGRA)  #Convert to RGBA (RGB with Alpha Channel)
        sign_rgba[:,:,3] = mask                             #Add mask to alpha channel
        # print(img_sign.shape) #(287, 124, 3)
        # cv2.drawContours(sign_rgba, valid_contours, -1, (0,255,0), 2)
        # cv2.imshow('Sign', sign_rgba)
        # cv2.waitKey(0)

        if not templates_dict.get(sign_type):  #Add new tuple to dictionary if it doesn't exist
            templates_dict[sign_type] = [(sign_number, sign_rgba, contours_list)]
        else:                                  #The tuples contains the sign number (_XXX), sign image -> RGBA: (XXX, XXX, 4)) and contours list [shape: (n, 1, 2)]]                                 
            templates_dict[sign_type].append((sign_number, sign_rgba, contours_list))

    return templates_dict
        
templates_dict = createTemplate(orange_signs_list)

In [4]:
def addBlur(img):
    kernel = np.ones((5,5), np.float32)/25
    img = cv2.filter2D(img, -1, kernel)
    return img

def checkIntervals(a, b, c, d):
    if a < b and c < d:
        return True
    else:
        return False
        

def pasteTemplateIntoCocoImage(coco_img, templates_list, templates_dict):
    countErrors = 0
    numberOfSigns = np.random.randint(1, 5) #Random number of signs to be placed in the image
    daily_templates_list, frequent_templates_list, uncommom_templates_list = templates_list[0]
    chosen_list = []


    while numberOfSigns > 0:
        chosen_frequency = random.choices([daily_templates_list, frequent_templates_list, uncommom_templates_list], weights=[0.5, 0.3, 0.2], k=1)[0]
        chosen_sign = random.choices(chosen_frequency, k=1)[0]
        chosen_sign = chosen_sign[:-4:] # remove .png
        chosen_list.append(chosen_sign)
        numberOfSigns -= 1

    # Regions: [ { shape_attributes
    #               { 
    #                   name:, all_points_x:, all_points_y:,
    #               },  
    #              region_attributes: 
    #              {    
    #                   class:
    #               } 
    #              }, 
    #            { shape_attributes
    #               { 
    #                   name:, all_points_x:, all_points_y:,
    #               },  
    #              region_attributes: 
    #              {    
    #                   class:
    #               } 
    #              }, ...]

    regions = []
        
    for i, sign in enumerate(chosen_list):
        sign_number = int(sign[-3::]) # select the number of this sign type
        sign_type = sign[:-4:] # select the sign type

        template_tuple = templates_dict[sign_type][sign_number]       
        template = template_tuple[1]
        temp_h, temp_w, _ = template.shape

        #print(template.shape)
        if i == 0:
            valid_intervals = checkIntervals(temp_w, WIDTH/2-temp_w, temp_h, HEIGHT/2-temp_h)
            if valid_intervals:
                x_offset, y_offset= np.random.randint(temp_w, WIDTH/2-temp_w), np.random.randint(temp_h, HEIGHT/2-temp_h)     #Change these to change the position of the sign
            else:
                x_offset, y_offset= np.random.randint(10, 20), np.random.randint(10, 20)     
        elif i == 1:
            valid_intervals = checkIntervals(WIDTH/2+temp_w, WIDTH - temp_w, temp_h, HEIGHT/2-temp_h)
            if valid_intervals:
                x_offset, y_offset= np.random.randint(WIDTH/2 + temp_w, WIDTH - temp_w), np.random.randint(temp_h, HEIGHT/2-temp_h)     #Change these to change the position of the sign
            else:
                x_offset, y_offset= np.random.randint(WIDTH/2 + 10, WIDTH/2 + 20), np.random.randint(10, 20)     
        elif i == 2:
            valid_intervals = checkIntervals(temp_w, WIDTH/2 - temp_w, HEIGHT/2+temp_h, HEIGHT - temp_h)
            if valid_intervals:
                x_offset, y_offset= np.random.randint(temp_w, WIDTH/2 - temp_w), np.random.randint(HEIGHT/2 + temp_h, HEIGHT - temp_h)     #Change these to change the position of the sign
            else:
                x_offset, y_offset= np.random.randint(10, 20), np.random.randint(HEIGHT/2 + 10, HEIGHT/2 + 20)
        else:
            valid_intervals = checkIntervals(WIDTH/2+temp_w, WIDTH - temp_w, HEIGHT/2+temp_h, HEIGHT - temp_h)
            if valid_intervals:
                x_offset, y_offset= np.random.randint(WIDTH/2+temp_w, WIDTH - temp_h), np.random.randint(HEIGHT/2+temp_h, HEIGHT-temp_h)     #Change these to change the position of the sign
            else:
                x_offset, y_offset= np.random.randint(WIDTH/2+10, WIDTH/2 + 20), np.random.randint(HEIGHT/2+10, HEIGHT/2 + 20)

        #Using the offset to drag the image down and to the left    
        y1, y2 = y_offset, y_offset + template.shape[0]
        x1, x2 = x_offset, x_offset + template.shape[1]

        alpha_s = template[:, :, 3] / 255.0
        alpha_l = 1.0 - alpha_s
        
        for c in range(0, 3):
            try:
                coco_img[y1:y2, x1:x2, c] = (alpha_s * template[:, :, c] +
                                    alpha_l * coco_img[y1:y2, x1:x2, c])
            except:
                countErrors+= 1
                continue   
        
        
    return coco_img, countErrors, regions

def createSampleImages(templates_dict, coco_imgs_list, templates_list, blurry, save, annotations):
    img_list = []
    totalErrors = 0

    for name in coco_imgs_list:
        name = name[:-4:] # remove .jpg
        coco_img = cv2.imread(f'./coco_dataset/images/chosenImages/{name}.jpg')
        if(coco_img.shape[0] > coco_img.shape[1]):    #Eliminate images with height greater than width (portraits)   
            continue

        coco_img = cv2.resize(coco_img, (WIDTH, HEIGHT), interpolation = cv2.INTER_LINEAR)

        coco_img, errors, regions_obj = pasteTemplateIntoCocoImage(coco_img, templates_list, templates_dict)
        totalErrors += errors        
        
        if blurry:
            coco_img = addBlur(coco_img)
            
        if save:
            cv2.imwrite(f'./Img/ArtificialSamples/{name}.png', coco_img)
            img_size = os.stat(f'./Img/ArtificialSamples/{name}.png').st_size
            if annotations:
                
                regions_obj = {
                    "shape_attributes": {
                        "name": "polygon",
                        "all_points_x": [],
                        "all_points_y": []
                    }, 
                    "region_attributes": {
                        "class": 
                    }
                }
                img_annotation = {name+'.jpg'+img_size: 
                    {
                        "file_name": name+'.jpg',
                        "size": img_size,
                        "regions":
                    }
                }
        
                 
        img_list.append(coco_img)
    print(f'Total errors: {totalErrors}')

    # cv2.imshow('Img', img_list[0])
    # cv2.waitKey(0)

createSampleImages(templates_dict, coco_imgs_list, templates_list, blurry=True, save=True, annotations=True)

SyntaxError: invalid syntax (<ipython-input-4-0a61f0f43db2>, line 130)